In [1]:
import glob, os, re
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split

In [2]:

df = pd.read_csv('corpus_f.csv') 
df.col=df.col.astype(str)

corpus = list(df["col"])
corpus

['<start> 무얼 믿 은 걸까 부족 했 던 내게서 <end>',
 '<start> 나 조차 못 믿 던 내게 여태 머문 사람 <end>',
 '<start> 무얼 봤 던 걸까 가진 것 도 없 던 내게 <end>',
 '<start> 무 작정 내 손 을 잡 아 날 이 끈 사람 <end>',
 '<start> 최고 였 어 <end>',
 '<start> 그대 눈 속 에 비친 내 모습 <end>',
 '<start> 이제 는 내게서 그댈 비춰 줄 게 <end>',
 '<start> 궂 은 비 가 오 면 <end>',
 '<start> 세상 가장 큰 그대 우산 이 될 게 <end>',
 '<start> 그댄 편히 걸어가 요 <end>',
 '<start> 걷 다가 지치 면 <end>',
 '<start> 내 가 그대 를 안 고 어디 든 갈 게 <end>',
 '<start> 이제 나 만 믿 어요 <end>',
 '<start> 나 만 두 고 가 던 <end>',
 '<start> 나 만 스쳐 간 행운 이 모여 <end>',
 '<start> 그대 가 되 어서 내게 와준 거 야 <end>',
 '<start> 궂 은 비 가 오 면 <end>',
 '<start> 세상 가장 큰 그대 우산 이 될 게 <end>',
 '<start> 그댄 편히 걸어가 요 <end>',
 '<start> 걷 다가 지치 면 <end>',
 '<start> 내 가 그대 를 안 고 어디 든 갈 게 <end>',
 '<start> 이제 나 만 믿 어요 <end>',
 '<start> 나 의 마지막 주인공 이 되 어 <end>',
 '<start> 다신 누구 앞 에서 도 <end>',
 '<start> 그대 는 고개 숙이 지 마요 <end>',
 '<start> 내 가 보 지 못했 던 홀로 고단 했 던 시간 <end>',
 '<start> 고맙 고 미안 해요 사랑 해요 <end>',
 '<start> 이 세상 은 <end>',
 '<start> 우리 를 두 고 오랜 장난 을 했 고 <end>',


In [3]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 12000, filters = ' ', oov_token = '<unk>')
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post')
    print('토크나이저: ', tokenizer, '\n', tensor)
    
    return tensor, tokenizer

In [4]:
tensor, tokenizer = tokenize(corpus)

토크나이저:  <keras.src.legacy.preprocessing.text.Tokenizer object at 0x000002B2CED42C90> 
 [[  2 764 176 ...   0   0   0]
 [  2   8 641 ...   0   0   0]
 [  2 764 603 ...   0   0   0]
 ...
 [  2  13 103 ...   0   0   0]
 [  2   4 419 ...   0   0   0]
 [  2  34   5 ...   0   0   0]]


In [5]:
tensor.shape

(125691, 76)

In [6]:
tensor[0]

array([   2,  764,  176,   11,  995, 1332,   69,   39, 2586,    3,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [8]:
for idx in tokenizer.index_word:
    print(idx, ':', tokenizer.index_word[idx])
    if idx>=30: break

1 : <unk>
2 : <start>
3 : <end>
4 : 이
5 : 는
6 : 을
7 : 고
8 : 나
9 : 사랑
10 : 에
11 : 은
12 : 가
13 : 내
14 : 아
15 : 도
16 : 어
17 : 하
18 : 의
19 : 지
20 : 한
21 : 를
22 : 야
23 : 있
24 : 게
25 : 그
26 : 다
27 : 당신
28 : 만
29 : 면
30 : 없


In [18]:
for idx in tokenizer.index_word:
    print(idx, ':', tokenizer.index_word[idx])
    if idx>=20: break

1 : <unk>
2 : <start>
3 : <end>
4 : 는
5 : 이
6 : 을
7 : 가
8 : 고
9 : 나
10 : 어
11 : 은
12 : 너
13 : 에
14 : 지
15 : 게
16 : 도
17 : 내
18 : 의
19 : 를
20 : 아


In [19]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]
print('텐서 길이: ', tensor.shape)
print('소스문장 길이: ', len(src_input[0]))
print('타겟문장 길이: ', len(tgt_input[0]))

텐서 길이:  (210237, 30)
소스문장 길이:  29
타겟문장 길이:  29


In [20]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 1234)
print('Source Train 길이: ', enc_train.shape)
print('Target Train 길이: ', dec_train.shape)
print('Source Test 길이: ', enc_val.shape)
print('Target Test 길이: ', dec_val.shape)

Source Train 길이:  (168189, 29)
Target Train 길이:  (168189, 29)
Source Test 길이:  (42048, 29)
Target Test 길이:  (42048, 29)


In [21]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epochs = len(src_input) // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words + 1

dataset_train = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset_train = dataset_train.shuffle(BUFFER_SIZE)
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder = True)

dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_val = dataset_val.shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder = True)

print(dataset_train)
print(dataset_val)

<_BatchDataset element_spec=(TensorSpec(shape=(256, 29), dtype=tf.int32, name=None), TensorSpec(shape=(256, 29), dtype=tf.int32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(256, 29), dtype=tf.int32, name=None), TensorSpec(shape=(256, 29), dtype=tf.int32, name=None))>


In [22]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [23]:
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)
model

<TextGenerator name=text_generator, built=False>

In [24]:
for src_sample, tgt_sample in dataset_train.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 29, 12001), dtype=float32, numpy=
array([[[ 2.33086117e-04, -4.37515337e-05,  1.28153813e-04, ...,
         -6.65189873e-05,  2.36103006e-07, -1.67994731e-05],
        [ 4.17938310e-04,  2.12294137e-04,  3.98943783e-04, ...,
          9.50473041e-05, -4.41949214e-06, -4.94528140e-05],
        [ 8.87059665e-04,  1.04523766e-04,  5.45665796e-04, ...,
          1.00166500e-04, -1.87115638e-05, -1.62265322e-04],
        ...,
        [-1.50751404e-03,  1.81679963e-03, -2.35585216e-03, ...,
          1.30219804e-03,  3.42019089e-03,  1.68479746e-03],
        [-1.54658384e-03,  1.85402844e-03, -2.28630216e-03, ...,
          1.28840201e-03,  3.43419844e-03,  1.69145397e-03],
        [-1.57898583e-03,  1.88547804e-03, -2.22368119e-03, ...,
          1.27311680e-03,  3.44486255e-03,  1.69567554e-03]],

       [[ 2.33086117e-04, -4.37515337e-05,  1.28153813e-04, ...,
         -6.65189873e-05,  2.36103006e-07, -1.67994731e-05],
        [ 5.97084698e-04, -1.84328019e-05,  3

In [25]:
model.summary()

Model: "text_generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     3,072,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │     5,246,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │     8,392,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │    12,301,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,012,961 (110.68 MB)

 Trainable params: 29,012,961 (110.68 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = 'none')

In [24]:
model.compile(loss = loss, optimizer = optimizer)
model.fit(dataset_train, epochs = 10)

Epoch 1/10


  1/656 ━━━━━━━━━━━━━━━━━━━━ 26:23:40 145s/step - loss: 9.3930

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=30):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> ", max_len=20)